# Fitting with polynomials

For our final fitting tutorial, we'll review how to fit opacity functions with polynomials. This approach is generally the least performant of the three currently supported, though it likely performs well enough for very smoothly varying opacity functions (such as collisionally induced absorption).

## Setting up the objects

In [ ]:
import numpy as np
import sys
import os

sys.path.insert(0, os.path.abspath("../../src"))

import cortecs
from cortecs.opac.opac import *
from cortecs.fit.fit import *
from cortecs.fit.fit_pca import *
from cortecs.eval.eval import *

We'll be using the same `Opac` object as in the Quickstart.

In [ ]:
T_filename = "temperatures.npy"
P_filename = "pressures.npy"
wl_filename = "wavelengths.npy"

cross_sec_filename = "absorb_coeffs_C2H4.npy"

load_kwargs = {
    "T_filename": T_filename,
    "P_filename": P_filename,
    "wl_filename": wl_filename,
}
opac_obj = Opac(cross_sec_filename, loader="platon", load_kwargs=load_kwargs)

Now, we instantiate a `Fitter` with the `polynomial` method.

In [ ]:
fitter = Fitter(opac_obj, method="polynomial")
fitter

In [ ]:
fitter.fit()

Let's use an `Evaluator` to see how well we captured the opacity function.

In [ ]:
evaluator = Evaluator(opac_obj, fitter)

In [ ]:
temperature = 300.0
pressure = 100
wavelength = 2.99401875e-05

evaluator.eval(pressure, temperature, wavelength)

Accuracy-wise, this doesn't perform that well. This performance hit is partially because this opacity zeros out in many corners of parameter space, and the polynomial fit can't account for those sharp edges.

Let's check the evaluation time, as well.

In [ ]:
%%timeit

evaluator.eval(temperature, pressure, wavelength)

In [ ]:
vals, orig_vals, abs_diffs, percent_diffs = calc_metrics(
    fitter, tp_undersample_factor=2, plot=True
);

This is a bit slower than some of the other methods, too.

In [ ]:
np.median(np.abs(abs_diffs))

In [ ]:
opac_obj.cross_section.nbytes / fitter.fitter_results[1].nbytes